<a href="https://colab.research.google.com/github/ellen008/DeepL/blob/master/Differential_Privacy_Project_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import numpy as np
import torchvision.datasets as datasets
from torchvision import datasets, transforms
from torch.utils.data import random_split, SubsetRandomSampler
from torch import nn, optim
import torch.nn.functional as F

In [0]:
#download the data

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(),download = True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

#mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)


# Datasets (data + labels):

In [0]:
train_dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train

In [0]:
test_dataset

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test

In [0]:
  #++++++++++++
  #+++++++++split MNIST train dataset on train.train/train.valid

  #+++++++++create 10 teachers' datasets from train.train //train_ds//
  #+++++++++split 10 teachers' train.train dataset in train/valid
  #output:
  #teacher_train_loader = [1 ... 10]
  #teacher_valid_loader = [1 ... 10]
    
  #++++++++++++++++++++++++++
  #+++++++++train_with_teacher() 10 models on 10 splitted trainsets
   
#TODO   
  #model.forward(train.valid_dataset) //valid_ds//
  #return 10 sets of true_labels = []  
    
#TODO
  #dp_labels = true_labels + laplacianM

#TODO
  #split train.valid_dataset and true_labels on train/valid
  #train.valid.train_loader
  #train.valid.valid_loader

  #train_model(train.valid_dataset + dp_labels)

#TODO
  #predict(MNIST test_dataset)
  #compare predictions accuracy for true labels and dp_labels
    

In [0]:
#split train dataset into train.train = train_ds/ train.valid = valid_ds

train_size = int(5 * len(train_dataset)/6)
valid_size = len(train_dataset) - train_size

train_ds, valid_ds = torch.utils.data.random_split(train_dataset, [train_size, valid_size])

print(f"Train.train dataset = {len(train_ds)}",
      f"\nTrain.valid dataset= {len(valid_ds)}")

Train.train dataset = 50000 
Train.valid dataset= 10000


In [0]:
#batch_size = 64

trainloader = torch.utils.data.DataLoader(dataset=train_ds, batch_size=64,shuffle = True)
testloader = torch.utils.data.DataLoader(dataset=valid_ds, batch_size=64,shuffle = True)

# Create model

In [0]:
class Conv(nn.Module):
    
    def __init__(self):
        super(Conv, self).__init__()
        self.conv1 = nn.Conv2d(1,10,kernel_size = 5)
        self.conv2 = nn.Conv2d(10,20, kernel_size = 5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self,x):
        
        x = F.max_pool2d(self.conv1(x),2)
        x = F.relu(x)
        x = F.max_pool2d(self.conv2_drop(self.conv2(x)),2)
        x = F.relu(x)
        x = x.view(-1,320)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, training = self.training)
        x = self.fc2(x)
        x = F.log_softmax(x,dim = 1)
        
        return x

In [0]:
#model = Classifier_1()
#model = Classifier_2()
model = Conv()
model

Conv(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

In [0]:
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

In [0]:
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

In [0]:

#reset the model
def reset_model_conv():
    model.conv1.weight.data.uniform_(0.0, 1.0)
    model.conv1.bias.data.fill_(0)
    model.conv2.weight.data.uniform_(0.0, 1.0)
    model.conv2.bias.data.fill_(0)
    model.fc1.weight.data.uniform_(0.0, 1.0)
    model.fc1.bias.data.fill_(0)
    model.fc2.weight.data.uniform_(0.0, 1.0)
    model.fc2.bias.data.fill_(0)
    return -1

# train model_1

In [0]:
#reset conv network:
reset_model_conv()

-1

In [0]:
def evaluate_model_1(device,criterion,optimizer):
    
    #device = device
    test_loss = 0
    acc = 0

    model.eval()


    with torch.no_grad():
                
         for inputs, labels in testloader:

            #inputs = inputs.view(inputs.shape[0], -1)
            inputs, labels = inputs.to(device), labels.to(device)

            log_ps = model.forward(inputs)

            #print(labels.shape,log_ps.shape)
            test_loss += criterion(log_ps, labels).item()

            ps = torch.exp(log_ps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            acc += torch.mean(equals.type(torch.FloatTensor))

            #test_count +=1
            
    return test_loss, acc       


In [0]:
def train_model_1():
    
    device = 'cuda'
    epochs = 10
    count = 0

    model.to(device)

    criterion = nn.NLLLoss()
    #optimizer = optim.Adam(model.parameters(),lr = 0.01)
    optimizer = optim.SGD(model.parameters(),lr= 0.01, momentum = 0.5)
    
    train_losses = []
    test_losses = []
    accuracy =  []
    
    for e in range(epochs):
        model.train()
        running_loss = 0
        train_count = 0

        for images, labels in trainloader:
            # Flatten MNIST images into a 784 long vector
            #images = images.view(images.shape[0], -1)

            images, labels = images.to(device), labels.to(device)

            # TODO: Training pass
            optimizer.zero_grad()

            #output = model(images)
            output = model.forward(images)

            #print(output.shape, labels.shape)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            #print(train_count)
            train_count+=1
            
        else:
            #model evaluation
            test_loss,acc = evaluate_model_1(device,criterion,optimizer)

            #collect accuracy, train_losses and test_losses for each epoch
            train_losses.append(running_loss/len(trainloader))
            test_losses.append(test_loss/len(testloader))
            accuracy.append(acc/len(testloader))
        
            print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Training Loss: {:.6f}.. ".format(train_losses[e]),
                  "Test Loss: {:.6f}.. ".format(test_losses[e]),
                  "Test Accuracy: {:.6f}".format(accuracy[e])
                  )
            
        
    return -1





In [0]:
 train_model_1()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch: 1/15..  Training Loss: 1.481831..  Test Loss: 0.420661..  Test Accuracy: 0.876990
Epoch: 2/15..  Training Loss: 0.580260..  Test Loss: 0.250757..  Test Accuracy: 0.924363
Epoch: 3/15..  Training Loss: 0.446941..  Test Loss: 0.186798..  Test Accuracy: 0.944367
Epoch: 4/15..  Training Loss: 0.378882..  Test Loss: 0.156817..  Test Accuracy: 0.950935
Epoch: 5/15..  Training Loss: 0.343157..  Test Loss: 0.144729..  Test Accuracy: 0.955115
Epoch: 6/15..  Training Loss: 0.315194..  Test Loss: 0.127703..  Test Accuracy: 0.960191
Epoch: 7/15..  Training Loss: 0.292774..  Test Loss: 0.119321..  Test Accuracy: 0.963973
Epoch: 8/15..  Training Loss: 0.278124..  Test Loss: 0.107984..  Test Accuracy: 0.968451
Epoch: 9/15..  Training Loss: 0.261998..  Test Loss: 0.098294..  Test Accuracy: 0.969447
Epoch: 10/15..  Training Loss: 0.253352..  Test Loss: 0.093691..  Test Accuracy: 0.971835
Epoch: 11/15..  Training Loss: 0.242978..  Test Loss: 0.092988..  Test Accuracy: 0.973328
Epoch: 12/15..  Tra

-1

# Let's divide train data into 10 private datasets :

In [0]:
#num_teachers = int(len(train_dataset)/len(test_dataset)) # we're working with  private datasets
num_examples = len(train_ds) # the size of OUR dataset
num_labels = 10 # number of lablels for our classifier

batch_size = 64

max_ind= num_examples   #50000
num_teachers = 10
teacher_subset = int(max_ind/num_teachers)

print(f"max_ind = {max_ind}",f"teacher_subset = {teacher_subset}")



max_ind = 50000 teacher_subset = 5000


In [0]:
#print(len(test_dataset))
#test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size=1)
#test_loader

#train_loader10 = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=teacher_subset, shuffle=True)

In [0]:
def split_train_ds():

    max_ind= num_examples  #50000
    teacher_len = int(max_ind/num_teachers)

    

    teacher_train_loader = []
    teacher_valid_loader = []

    start = 0
    stop = teacher_len                          #5000
    train_size = int(0.8* teacher_len)          #4000
    test_size = teacher_len - train_size        #1000
    
    print(f"start = {start}",f"train_size = {train_size}",f"test_size = {test_size}")

    indicies = torch.randperm(max_ind)

    for i in range(num_teachers):
        idx =[j for j in range(start,stop)]
        idx = indicies[start:stop]
        
        #split teacher_j dataset into train/validation

        
        #train_teach_ds, test__teach_ds = torch.utils.data.random_split(train_dataset, [train_size, test_size])
        
        #split every teacher subset into train/valid
        train_t_idx=indicies[start:(start+train_size)]
        valid_t_idx = indicies[(start+train_size): stop]

        #print(f"Teacher Train dataset = {start+train_size}",
              #f"\nTeacher Valid dataset= {stop}")
    
        teacher_train_loader.append(torch.utils.data.DataLoader( train_ds, batch_size=batch_size, sampler = SubsetRandomSampler(train_t_idx)))
        teacher_valid_loader.append(torch.utils.data.DataLoader( train_ds, batch_size=batch_size, sampler = SubsetRandomSampler(valid_t_idx)))
        
        #print(f"teacher_train_loader = {len(teacher_train_loader[i])}",f"teacher_valid_loader = {len(teacher_valid_loader[i])}")
    
        start = stop
        stop = stop+teacher_len
        
    return teacher_train_loader, teacher_valid_loader


In [0]:
teacher_train_loader,teacher_valid_loader = split_train_ds()
print(len(teacher_train_loader), len(teacher_valid_loader))

start = 0 train_size = 4000 test_size = 1000
10 10


# Train model:

In [0]:
#reset_model_conv()

In [0]:
def evaluate_model_t(device,testloader_t,criterion):
    
    test_loss = 0
    acc = 0

    model.eval()


    with torch.no_grad():
                
         for inputs, labels in testloader_t:

            #inputs = inputs.view(inputs.shape[0], -1)
            inputs, labels = inputs.to(device), labels.to(device)

            log_ps = model.forward(inputs)

            #print(labels.shape,log_ps.shape)
            test_loss += criterion(log_ps, labels)

            ps = torch.exp(log_ps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            acc += torch.mean(equals.type(torch.FloatTensor))

            #test_count +=1
            
    return test_loss, acc       


In [0]:
def train_model_t(trainloader_t,testloader_t):
    
    device = 'cuda'
    epochs = 12
    count = 0

    model.to(device)

    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.parameters())
    
    #testloader = testloader
    
    train_losses = []
    test_losses = []
    accuracy =  []
    
    for e in range(epochs):
        model.train()
        running_loss = 0
        train_count = 0

        for images, labels in trainloader_t:
            # Flatten MNIST images into a 784 long vector
            #images = images.view(images.shape[0], -1)

            images, labels = images.to(device), labels.to(device)

            # TODO: Training pass
            optimizer.zero_grad()

            #output = model(images)
            output = model.forward(images)

            #print(output.shape, labels.shape)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            #print(train_count)
            train_count+=1
            
        else:
            #model evaluation
            test_loss,acc = evaluate_model_t(device,testloader_t,criterion)

            #collect accuracy, train_losses and test_losses for each epoch
            train_losses.append(running_loss/len(trainloader_t))
            test_losses.append(test_loss/len(testloader_t))
            accuracy.append(acc/len(testloader_t))
        
            print("Epoch: {}/{}.. ".format(e+1, epochs),
                  "Training Loss: {:.6f}.. ".format(train_losses[e]),
                  "Test Loss: {:.6f}.. ".format(test_losses[e]),
                  "Test Accuracy: {:.6f}".format(accuracy[e])
                  )
            
        
    return -1



In [0]:
for t in range(num_teachers):
    
    print(f"Teacher {t+1}")
    #instansiate next teacher model
    model = Conv()
    
    #train next teacher model
    train_model_t(teacher_train_loader[t],teacher_valid_loader[t])
    
    #re-initialize model parameters
    reset_model_conv()

Teacher 1
Epoch: 1/12..  Training Loss: 2.009555..  Test Loss: 1.130273..  Test Accuracy: 0.731836
Epoch: 2/12..  Training Loss: 1.034178..  Test Loss: 0.535665..  Test Accuracy: 0.855859
Epoch: 3/12..  Training Loss: 0.722373..  Test Loss: 0.396389..  Test Accuracy: 0.883398
Epoch: 4/12..  Training Loss: 0.612483..  Test Loss: 0.350082..  Test Accuracy: 0.900195
Epoch: 5/12..  Training Loss: 0.522144..  Test Loss: 0.313830..  Test Accuracy: 0.907031
Epoch: 6/12..  Training Loss: 0.487215..  Test Loss: 0.285592..  Test Accuracy: 0.920117
Epoch: 7/12..  Training Loss: 0.448696..  Test Loss: 0.263580..  Test Accuracy: 0.926562
Epoch: 8/12..  Training Loss: 0.410505..  Test Loss: 0.262283..  Test Accuracy: 0.922656
Epoch: 9/12..  Training Loss: 0.411605..  Test Loss: 0.251336..  Test Accuracy: 0.926953
Epoch: 10/12..  Training Loss: 0.357945..  Test Loss: 0.243183..  Test Accuracy: 0.923828
Epoch: 11/12..  Training Loss: 0.357115..  Test Loss: 0.232707..  Test Accuracy: 0.929883
Epoch: 12

# Snippets

In [0]:
images = []
labels = []
j=0

for i in range(len(train_targets)):
    
    images.append(train_data[i])
    labels.append(train_targets[i])
    
    
    
    if (i+1)%6000==0:
        print ("Teacher ", j)
        print(len(images),len(labels))
        images.clear()
        labels.clear() 
        j+=1


In [0]:
images = []
labels = []
image_folder=[]
label_folder=[]
j=0

for i in range(len(train_targets)):
    
    images.append(train_data[i])
    labels.append(train_targets[i])
    
    if (i+1)%6000==0:
        print (j)
        image_folder.append(images) 
        #label_folder[j] = labels
        
        #images = []
        #labels = []
        j+=1


In [0]:
class Classifier_1(nn.Module):
    def __init__(self):
        super().__init__()
        # Defining the layers, 128, 64, 10 units each
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 64)
        # Output layer, 10 units - one for each digit
        self.fc4 = nn.Linear(64, 10)
        
    def forward(self, x):
        
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.fc4(x)
        x = F.log_softmax(x, dim=1)
        
        return x
      

In [0]:
#reset the model
def reset_model_2():
    model.fc1.weight.data.uniform_(0.0, 1.0)
    model.fc1.bias.data.fill_(0)
    model.fc2.weight.data.uniform_(0.0, 1.0)
    model.fc2.bias.data.fill_(0)
    model.fc3.weight.data.uniform_(0.0, 1.0)
    model.fc3.bias.data.fill_(0)
    model.fc4.weight.data.uniform_(0.0, 1.0)
    model.fc4.bias.data.fill_(0)
    return -1

In [0]:
class Classifier_2(nn.Module):
    def __init__(self):
        super().__init__()
        # Defining the layers, 128, 64, 10 units each
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        # Output layer, 10 units - one for each digit
        self.fc5 = nn.Linear(64, 10)
        
    def forward(self, x):
        
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.fc4(x)
        x = F.relu(x)
        x = self.fc5(x)
        x = F.log_softmax(x, dim=1)
        
        return x
      

In [0]:
class Teacher():
  
  def __init__(self, loader, dest)
  
    self.image_loader = loader
    self.destination = dest
    
    batch_size = 60
    #n_iters = 1000
    #epochs = n_iters / (len(train_dataset) / batch_size)
    
    self.criterion = nn.NLLLoss()
    self.optimaizer = optim.Adam(model.parameters(), lr=0.003)
    
    
  def train(self):
    
    model.to('cuda')


    device = 'cuda'
    epochs = 5
    steps = 0

    train_losses, test_losses = [], []
    
    
    for e in range(epochs):
      
        running_loss = 0
        accuracy = 0
        
        for inputs, labels in trainloader:
        
            #model.train()
            optimizer.zero_grad()
        
            inputs, labels = inputs.to(device), labels.to(device)
        
            preds = model.forward(inputs)
            loss = criterion(preds, labels)
            loss.backward()
            optimizer.step()
        
            running_loss += loss.item()
        
        else:
            test_loss = 0
            accuracy = 0
        
            model.eval()
        
            # Turn off gradients for validation, saves memory and computations
            with torch.no_grad():
                for inputs, labels in testloader:
              
                    inputs, labels = inputs.to(device), labels.to(device)
                
                    log_ps = model.forward(inputs)
                    test_loss += criterion(log_ps, labels)
                
                    ps = torch.exp(log_ps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor))
                
            train_losses.append(running_loss/len(trainloader))
            test_losses.append(test_loss/len(testloader))
        
        running_loss = 0
        model.train()

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))
      
      
    return accuracy
      
      
  def get_labels():
    
    return true_labels
    
    

In [0]:
images, labels = next(iter(testloader))
images = images.view(images.shape[0], -1)

# Clear the gradients, do this because gradients are accumulated
optimizer.zero_grad()

# Forward pass, then backward pass, then update weights
output = model(images)
print(output.shape,labels.shape)

torch.Size([64, 10]) torch.Size([64])


In [0]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        # Defining the layers, 784, 256, 10 units each
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 64)
        # Output layer, 10 units - one for each digit
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, x):
        
        x = self.fc1(x)
        x = F.relu(x)
        #print(x.shape)
        
        x = self.fc2(x)
        x = F.relu(x)
        #print(x.shape)
        
        x = self.fc3(x)
        #print(x.shape)
        
        x = F.log_softmax(x, dim=1)            #softmax
        #print(x.shape)
        
        return x
      

In [0]:
model = Classifier()
model

Classifier(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)

In [0]:
print("Model's state_dict:")
for param_tensor in model.state_dict():
    
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())


Model's state_dict:
fc1.weight 	 torch.Size([256, 784])
fc1.bias 	 torch.Size([256])
fc2.weight 	 torch.Size([64, 256])
fc2.bias 	 torch.Size([64])
fc3.weight 	 torch.Size([10, 64])
fc3.bias 	 torch.Size([10])


In [0]:
#reset the model
def reset_model_1():
    model.fc1.weight.data.uniform_(0.0, 1.0)
    model.fc1.bias.data.fill_(0)
    model.fc2.weight.data.uniform_(0.0, 1.0)
    model.fc2.bias.data.fill_(0)
    model.fc3.weight.data.uniform_(0.0, 1.0)
    model.fc3.bias.data.fill_(0)
    return -1

In [0]:
params = list(model.parameters())
print(len(params))
#params

6


In [0]:
#batch_size = 64

trainloader = torch.utils.data.DataLoader(dataset=train_ds, batch_size=64,shuffle = True)
testloader = torch.utils.data.DataLoader(dataset=valid_ds, batch_size=64,shuffle = True)


In [0]:
#train the model on the whole 'MNIST' to tune hyperparameter

device = 'cuda'
epochs = 7
count = 0

model.to(device)

criterion = nn.NLLLoss()
#optimizer = optim.SGD(model.parameters(), lr=0.01)
optimizer = optim.Adam(model.parameters(), lr = 0.01)


for e in range(epochs):
  
    model.train()
    running_loss = 0
    train_count = 0
    
    for images, labels in trainloader:
        # Flatten MNIST images into a 784 long vector
        images = images.view(images.shape[0], -1)
        
        images, labels = images.to(device), labels.to(device)
    
        # TODO: Training pass
        optimizer.zero_grad()
        
        #output = model(images)
        output = model.forward(images)
        
        #print(output.shape, labels.shape)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        #print(train_count)
        train_count+=1
  
    else:
        test_loss = 0
        accuracy = 0
        
        model.eval()
        
            
        with torch.no_grad():
            #test_count =0
            for inputs, labels in testloader:
                
                inputs = inputs.view(inputs.shape[0], -1)
                inputs, labels = inputs.to(device), labels.to(device)
                
                log_ps = model.forward(inputs)
                
                #print(labels.shape,log_ps.shape)
                test_loss += criterion(log_ps, labels)
                
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
                
                #test_count +=1
                
            print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.6f}.. ".format(running_loss/len(trainloader)),
              "Test Loss: {:.6f}.. ".format(test_loss/len(testloader)),
              "Test Accuracy: {:.6f}".format(accuracy/len(testloader))
              )
    
    #collect train_losses and test_losses for each epoch
    #train_losses.append(running_loss/len(trainloader))
    #test_losses.append(test_loss/len(testloader))
        
     


Epoch: 1/7..  Training Loss: 363.900989..  Test Loss: 6.337336..  Test Accuracy: 0.871218
Epoch: 2/7..  Training Loss: 3.837848..  Test Loss: 2.177621..  Test Accuracy: 0.903364
Epoch: 3/7..  Training Loss: 1.061672..  Test Loss: 0.714465..  Test Accuracy: 0.934415
Epoch: 4/7..  Training Loss: 0.374929..  Test Loss: 0.459030..  Test Accuracy: 0.943770
Epoch: 5/7..  Training Loss: 0.200605..  Test Loss: 0.259740..  Test Accuracy: 0.955215
Epoch: 6/7..  Training Loss: 0.121275..  Test Loss: 0.255535..  Test Accuracy: 0.958002
Epoch: 7/7..  Training Loss: 0.113166..  Test Loss: 0.241711..  Test Accuracy: 0.954021
